<a href="https://colab.research.google.com/github/manikantaaaaaaaa/Qml_Task/blob/main/QML_Task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TASK 4: Commutation Relations and Euler Decomposition**

### **Aim**
To verify the **Pauli matrix commutation relations** and to perform **Z–Y–Z Euler angle decomposition** for arbitrary single-qubit gates.



### **1. Mathematical Model**

#### **1.1 Pauli Matrix Commutation & Anti-Commutation Relations**

The Pauli matrices are defined as:

\
$X =
\begin{bmatrix}
0 & 1 \\
1 & 0
\end{bmatrix}, \quad
Y =
\begin{bmatrix}
0 & -i \\
i & 0
\end{bmatrix}, \quad
Z =
\begin{bmatrix}
1 & 0 \\
0 & -1
\end{bmatrix}
$

The **commutator** of two operators \(A\) and \(B\) is given by:

\[
[A, B] = AB - BA
\]

For Pauli matrices:

\[$
[\sigma_i, \sigma_j] = 2i \varepsilon_{ijk} \sigma_k
$\]

where \($\varepsilon_{ijk}$\) is the **Levi-Civita symbol**.

Similarly, the **anti-commutator** is defined as:

\[
\{A, B\} = AB + BA
\]

For Pauli matrices:

\[$
\{\sigma_i, \sigma_j\} = 2 \delta_{ij} I
$\]

where \($\delta_{ij}$\) is the **Kronecker delta**, and \(I\) is the 2×2 identity matrix.

These relations confirm the fundamental algebraic properties of Pauli matrices in quantum mechanics.



#### **1.2 Z–Y–Z Euler Decomposition for Single-Qubit Gates**

Any single-qubit unitary \(U \in SU(2)\) can be expressed (up to a global phase \$(e^{i\phi}$\)) as:

\[$
U = e^{i\phi} R_z(\alpha) R_y(\beta) R_z(\gamma)
$\]

where:
- \(\phi\) is the **global phase**,  
- \(\alpha, \beta, \gamma\) are the **rotation angles**, and  
- the rotation operators are defined as:  

\
$
R_z(\theta) =
\begin{bmatrix}
e^{-i\theta/2} & 0 \\
0 & e^{i\theta/2}
\end{bmatrix}, \quad
R_y(\theta) =
\begin{bmatrix}
\cos(\theta/2) & -\sin(\theta/2) \\
\sin(\theta/2) & \cos(\theta/2)
\end{bmatrix}
$


**Decomposition Steps:**
1. Extract the global phase from the determinant of \(U\).  
2. Normalize \(U\) to remove the global phase.  
3. Solve for the Euler angles \(\alpha, \beta, \gamma\) based on the matrix elements of \(U\).  
4. Validate the decomposition by reconstructing the unitary matrix.



### **2. Algorithm**

#### **2.1 Pauli Matrix Verification**
- Define Pauli matrices symbolically using **SymPy**.  
- Compute commutators \([A,B] = AB - BA\) and verify \([\sigma_i, \sigma_j] = 2i\varepsilon_{ijk}\sigma_k\).  
- Compute anti-commutators \(\{A,B\} = AB + BA\) and verify \(\{\sigma_i, \sigma_j\} = 2\delta_{ij}I\).  

#### **2.2 Z–Y–Z Euler Decomposition**
- Check matrix unitarity: \(U^\dagger U = I\).  
- Extract the global phase from the determinant.  
- Solve for Euler angles \((\alpha, \beta, \gamma)\) in \(U = e^{i\phi}R_z(\alpha)R_y(\beta)R_z(\gamma)\).  
- Handle special cases where \(\beta \approx 0\) or \(\pi\).  
- Reconstruct the unitary matrix to validate the decomposition.

#### **2.3 Testing**
- Validate decomposition for standard gates: **X, Y, Z, H, S, T**.  
- Test random unitary matrices for generalization.  
- Optionally integrate with **Cirq** for hardware-level verification.





In [ ]:
!pip install cirq
import numpy as np
import cmath
print("\n" + "="*50)
print("TASK 4: COMMUTATION RELATIONS AND EULER ANGLES")
print("="*50)
# --- Part 1: Verify Pauli commutation & anti-commutation with SymPy -
import sympy as sp
I = sp.eye(2)
sx = sp.Matrix([[0, 1], [1, 0]])
sy = sp.Matrix([[0, -sp.I], [sp.I, 0]])
sz = sp.Matrix([[1, 0], [0, -1]])
paulis = {'X': sx, 'Y': sy, 'Z': sz}
def comm(A, B):
 return sp.simplify(A * B - B * A)
def anti(A, B):
 return sp.simplify(A * B + B * A)
print("\n=== Commutation relations ===")
for (a, b, k) in [('X', 'Y', 'Z'), ('Y', 'Z', 'X'), ('Z', 'X', 'Y')]:
 lhs = comm(paulis[a], paulis[b])
 rhs = 2 * sp.I * paulis[k]
 print(f"[{a},{b}] =\n{lhs}\nExpected:{rhs}\n")
print("\n=== Anti-commutation relations ===")
for i in ['X', 'Y', 'Z']:
 for j in ['X', 'Y', 'Z']:
  lhs = anti(paulis[i], paulis[j])
  rhs = 2 * (1 if i == j else 0) * I
  print(f"{{{i},{j}}} =\n{lhs} Expected:{rhs}\n")
# --- Part 2: Z–Y–Z Euler decomposition ---
def is_unitary(U, tol=1e-8):
 return np.allclose(U.conj().T @ U, np.eye(2), atol=tol)
def decompose_zyz(U, tol=1e-8):
 """Return (phi, alpha, beta, gamma) such that
 U = e^{i phi} Rz(alpha) Ry(beta) Rz(gamma)
 """
 U = np.array(U, dtype=complex)
 if not is_unitary(U):
  raise ValueError("Matrix is not unitary.")
 detU = np.linalg.det(U)
 phi = cmath.phase(detU) / 2
 U0 = U * np.exp(-1j * phi)
 detU0 = np.linalg.det(U0)
 U0 = U0 / np.sqrt(detU0)
 a = U0[0, 0]
 b = U0[0, 1]
 beta = 2 * np.arccos(min(1.0, max(0.0, abs(a))))
 if np.isclose(np.sin(beta / 2), 0, atol=tol):
  alpha = 2 * (-cmath.phase(a))
  gamma = 0.0
 else:
  phi1 = -cmath.phase(a)
  phi2 = -cmath.phase(-b)
  alpha = phi1 + phi2
  gamma = phi1 - phi2
 return float(phi), float(alpha), float(beta), float(gamma)
def Rz(theta):
 return np.array([[np.exp(-1j * theta / 2), 0],
 [0, np.exp(1j * theta / 2)]], dtype=complex)
def Ry(theta):
 return np.array([[np.cos(theta / 2), -np.sin(theta / 2)],
 [np.sin(theta / 2), np.cos(theta / 2)]],
dtype=complex)
def reconstruct(phi, alpha, beta, gamma):
 return np.exp(1j * phi) @ (Rz(alpha) @ Ry(beta) @ Rz(gamma))
# --- Part 3: Test examples ---
def Rx(theta):
 return np.cos(theta / 2) * np.eye(2) - 1j * np.sin(theta / 2) * sx
examples = {
 "Rx(pi/3)": Rx(np.pi / 3),
 "Ry(pi/4)": Ry(np.pi / 4),
 "Rz(pi/2)": Rz(np.pi / 2),
 "H": (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]],
dtype=complex),
 "S": np.array([[1, 0], [0, 1j]], dtype=complex),
 "T": np.array([[1, 0], [0, np.exp(1j * np.pi / 4)]],
dtype=complex),
}
print("\n=== Z–Y–Z Euler Decomposition ===")
for name, U in examples.items():
 phi, alpha, beta, gamma = decompose_zyz(U)
 print(f"{name}:\n φ={phi:.6f}, α={alpha:.6f}, β={beta:.6f},γ={gamma:.6f}\n")
# Optional: Use Cirq if available
try:
 import cirq
 print("\nCirq example decomposition for H gate:")
 # Create a qubit and turn H into an operation
 q = cirq.LineQubit(0)
 H_op = cirq.H(q)
 # Extract the unitary matrix of H
 U = cirq.unitary(H_op)
 # Perform Z–Y–Z decomposition
 phi, alpha, beta, gamma = decompose_zyz(U)
 print(f"Cirq H: φ={phi:.6f}, α={alpha:.6f}, β={beta:.6f}, γ={gamma:.6f}")
except ImportError:
 print("\nCirq not installed. Skipping Cirq examples.")


TASK 4: COMMUTATION RELATIONS AND EULER ANGLES

=== Commutation relations ===
[X,Y] =
Matrix([[2*I, 0], [0, -2*I]])
Expected:Matrix([[2*I, 0], [0, -2*I]])

[Y,Z] =
Matrix([[0, 2*I], [2*I, 0]])
Expected:Matrix([[0, 2*I], [2*I, 0]])

[Z,X] =
Matrix([[0, 2], [-2, 0]])
Expected:Matrix([[0, 2], [-2, 0]])


=== Anti-commutation relations ===
{X,X} =
Matrix([[2, 0], [0, 2]]) Expected:Matrix([[2, 0], [0, 2]])

{X,Y} =
Matrix([[0, 0], [0, 0]]) Expected:Matrix([[0, 0], [0, 0]])

{X,Z} =
Matrix([[0, 0], [0, 0]]) Expected:Matrix([[0, 0], [0, 0]])

{Y,X} =
Matrix([[0, 0], [0, 0]]) Expected:Matrix([[0, 0], [0, 0]])

{Y,Y} =
Matrix([[2, 0], [0, 2]]) Expected:Matrix([[2, 0], [0, 2]])

{Y,Z} =
Matrix([[0, 0], [0, 0]]) Expected:Matrix([[0, 0], [0, 0]])

{Z,X} =
Matrix([[0, 0], [0, 0]]) Expected:Matrix([[0, 0], [0, 0]])

{Z,Y} =
Matrix([[0, 0], [0, 0]]) Expected:Matrix([[0, 0], [0, 0]])

{Z,Z} =
Matrix([[2, 0], [0, 2]]) Expected:Matrix([[2, 0], [0, 2]])


=== Z–Y–Z Euler Decomposition ===
Rx(pi/3):
 φ=0

### **4. Result**

The commutation and anti-commutation properties of the **Pauli matrices** were successfully verified. The relations \([\sigma_i, \sigma_j] = 2i\varepsilon_{ijk}\sigma_k\) and \(\{\sigma_i, \sigma_j\} = 2\delta_{ij}I\) were observed to hold true.  

Additionally, the **Z–Y–Z Euler decomposition** for single-qubit gates was successfully implemented and validated. The decomposition accurately reconstructed standard quantum gates and confirmed that any single-qubit unitary can be expressed as a combination of Z and Y rotations up to a global phase. The experimental results align with theoretical expectations, confirming the correctness of the decomposition process.